### Experiment Runner 

Jeff Smith

Created: 2025-10-24

#### Setup and Connect to Portal

In [ ]:
#
# Creates api object and connects to Portal (parameters in .env)
#
# Imports - Make sure to delete the __pycache__ directory and restart the kernel if you change helper.py.
from helper import *
from pysimio import pySimio
from dotenv import load_dotenv
import os
import threading
from pysimio.classes import TimeOptions
import json
import pandas as pd
import datetime

# Load environment variables
# Makes sure that .env is set up for the portal instance that you want. The sample.env file has
# more details.
load_dotenv(override=True)
simio_portal_url = os.getenv("SIMIO_PORTAL_URL")
print(f"Connecting to portal on: {simio_portal_url}")
personal_access_token = os.getenv("PERSONAL_ACCESS_TOKEN")
auth_refresh_time = 500  # Time in seconds to refresh the auth token
run_status_refresh_time = 10
# Ensure token is loaded
if not personal_access_token:
    raise ValueError("Personal access token not found. Make sure it's set in the environment.")

# API Initialization getting bearer token for authorization
api = pySimio(simio_portal_url)
api.authenticate(personalAccessToken=personal_access_token)

# Start token refresh in a background thread
threading.Thread(target=refresh_auth_token, args=(api, auth_refresh_time), daemon=True).start()

Connecting to portal on: https://external.simioportal.com/
Refreshing authentication token...


Token refreshed successfully.


#### Getting Model/Experiment Information and Deleting Target Runs (if they exists) (optional)

In [2]:
#
# Gets the model, experiment, run information from Portal
# model_id and experiment_id are identified in this cell
#
# Project and Run names
project_name = "Optimization Example 03"
run_name = "API01"
# For sorting the scenarios after the run - These will be model-specific
sort_response = "Throughput"
sort_ascending = False

# Get Model ID for project by project name - Assumes that the project has a single model
# (will work if the project has multiple models, but this code return the "first" model in the project)
all_models_json = api.getModels()
model_id = find_modelid_by_projectname(all_models_json, project_name)
print(f"The model_id for project '{project_name}' is {model_id}")
all_experiments_json = api.getExperiments()
# Returns the "first" experiment for the model
experiment_id = find_id_by_model_id(all_experiments_json, model_id)
print(f"The first experiment_id for model {model_id} is {experiment_id}")
runs = get_runs_for_experiment(api, experiment_id)
if runs:
    print(f"Existing runs for {run_name}: {runs}.")
else:
    print(f"Run {run_name} does not exist for experiment {experiment_id}")

The model_id for project 'Optimization Example 03' is 108
The first experiment_id for model 108 is 174
Existing runs for API01: [419, 420, 421, 425, 426, 427].


In [ ]:
# manually setting the model and experiment (useful if you want
# an experiment that's not the first one for the model)
model_id = 104
experiment_id = 161

In [ ]:
#
# Delete existing runs for experiment
#
runs = get_runs_for_experiment(api, experiment_id)
if runs:
    delete_runs(api, runs)
    print(f"Deleted runs {runs}.")
else:
    print(f"No runs found for experiment {experiment_id}")

#### Create the Experiment Run JSON

In [ ]:
#
# experiment_id and run_name must be set before running this cell
#
# Experiment definition, experiment_def, is created here. 
#
# These are the experiment controls and values to enumerate over
# Specific to the example "Optimization Model 03.spfx"
vars = {
    "Buff1" : [3, 5],
    "Buff2" : [3, 5],
    "Buff3" : [5, 6],
    "CapA"  : [1, 2],
    "CapB"  : [3, 4],
    "CapC"  : [2, 3,4],
    "CapD"  : [2, 3]
}
num_reps = 6
experiment_def = create_ff_experiment(experiment_id, run_name, num_reps, vars)
n = len(experiment_def["CreateInfo"]["Scenarios"])
print(f"Experiment {experiment_def["Name"]} has {n} scenarios and {n*num_reps} total replications.")

#### Run the Created Experiment and Report

In [ ]:
# Run the experiment.
#
# Need these to be defined before executing this cell:
#   experiment_def - the experiment definition json
#   experiment_id
#   run_name
#
# Set to True if you want this cell to wait for run completion
WaitForRun = False

run_id = api.startRun(experiment_def)
all_runs_json = api.getRuns()
# Get the current date and time
current_time_local = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started run {run_name} ({run_id}) with {len(experiment_def["CreateInfo"]["Scenarios"])} scenarios at {current_time_local}.")
theRun = get_run(api, run_id, True)
if WaitForRun:
    sleep_time = 10 # seconds
    status = wait_for_run(api, run_id, sleep_time, 2000, False)
    print(f"Final run status after approx. {status[1]*sleep_time/60:.2f} minutes ({status[1]} cycles): {status[0]}")
else:
     print(f"Not waiting for run to finish.")


In [4]:
# Get the run Status - run_id must be set
#
theRun = get_run(api, run_id, True)


Status of run 427 at 2025-11-20 10:15: Running (8562/13122)


In [3]:
# Manually set run_id -- Shouldn't need to run this unless you want to 
# see the results from a different run than was done above.
run_id = 427
sort_response = "Throughput"
sort_ascending = False

In [ ]:
# Get the experiment response results
#
rows_to_show = 8
scenarios = api.getScenarios(run_id = run_id)
# Convert to dataframe
df, responses, controls = scenario_results_as_df(scenarios)
# remove any scenarios with non-numeric values in the sort column
df[sort_response] = pd.to_numeric(df[sort_response], errors='coerce')
df_cleaned = df.dropna(subset=[sort_response])
if len(df) > len (df_cleaned):
    print(f"Removed {len(df)-len(df_cleaned)} rows with non-numeric values in the sort column.")
print(f"Dataframe from run_id {run_id} includes {len(df_cleaned)} scenarios.")
# --- Display the Table ---
print(f"--- Top (up to) {rows_to_show} Scenario Response Averages by {sort_response} ---")
df_sorted = df_cleaned.sort_values(by=sort_response, ascending=sort_ascending)
print(df_sorted[:rows_to_show].to_string())

In [ ]:
# Show - In case you want to look at more results without rerunning.
#
rows_to_show = 100
print(f"Dataframe from run_id {run_id} includes {len(df_cleaned)} scenarios.")
# --- Display the Table ---
print(f"--- Top (up to) {rows_to_show} Scenario Response Averages by {sort_response} ---")
df_sorted = df_cleaned.sort_values(by=sort_response, ascending=sort_ascending)
print(df_sorted[:rows_to_show].to_string())